# Constant Bidder

## Environment

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

## Data Loader

In [12]:
data_path = os.path.abspath(os.pardir + '/Data/validation.csv')
df = pd.read_csv(data_path, na_values=['Na', 'null']).fillna(0)

### Data Exploration

In [6]:
df_clicks = df.loc[df["click"] == 1]
df_clicks.reset_index(drop=True, inplace=True)

print(df_clicks.mean())

feature = 'advertiser'
df_grouped_mean = df_clicks.groupby([feature]).mean()
df_grouped_sum = df_clicks.groupby([feature]).sum()

df_grouped_mean['eCPC'] = df_grouped_sum['payprice'] / (df_grouped_sum['click'])
df_grouped_mean['ctr'] = df_grouped_sum['click'] / df['advertiser'].value_counts()

#print(df_grouped['eCPC'], "\n")

df_clicks.mean()
#df_grouped_mean

df_overview = pd.DataFrame(df_grouped_mean, columns=['ctr', 'eCPC', 'payprice', 'bidprice'])
print(df_overview)

click            1.000000
weekday          2.854992
hour            14.539877
region         147.424986
city           152.046849
adexchange       1.786949
urlid            0.000000
slotwidth      555.631344
slotheight     175.156163
slotformat       0.551032
slotprice       31.026771
bidprice       273.226994
payprice       105.464027
advertiser    2846.489124
dtype: float64
                 ctr        eCPC    payprice    bidprice
advertiser                                              
1458        0.000782   96.576623   96.576623  300.000000
2259        0.000322  107.604651  107.604651  290.441860
2261        0.000327  132.000000  132.000000  287.388889
2821        0.000620  119.839695  119.839695  290.755725
2997        0.004355   89.036866   89.036866  277.000000
3358        0.000762  110.871287  110.871287  232.188119
3386        0.000703  118.737500  118.737500  300.000000
3427        0.000675  103.533088  103.533088  235.047794
3476        0.000602  101.406417  101.406417  248.8

### Incoming bids

target and feature variables

'target' : 'click',     # 'payprice', 'bidprice'   # select the target

'features': ['weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP', 'region', 'city', 'adexchange', 'domain', 
'url', 'urlid', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice', 'creative', 'keypage', 'advertiser', 'usertag'], # select the data features


In [14]:
#budget = 6250 * 1000
#constant_bid = 11

# 10, 1.  

## Evaluation Stats_____________

bids_won = 0
earned_clicks = 0
ctr = 0                  # bids_won / earned_clicks
total_paid = 0
cpc = 0                  # cost per click



constant_bids = np.arange(68,75,1)


#print("starting budget:", budget, "\n")

for constant_bid in constant_bids:
    
    ## shuffle dataframe rows
    df = df.sample(frac=1).reset_index(drop=True)
    
    print("\n\nconstant_bid:", constant_bid, "\n")
    
    ## Evaluation Stats_____________
    budget = 6250 * 1000

    bids_won = 0
    earned_clicks = 0
    ctr = 0                  # bids_won / earned_clicks
    total_paid = 0
    cpc = 0                  # cost per click

    for index, row in df.iterrows():

        if constant_bid > budget: # check if budget is sufficient for bidprice
            constant_bid = budget
            #print("constant bid reduced to:", constant_bid, ", total_paid:", total_paid, ", bids_won:", bids_won, ", earned clicks:", earned_clicks, "\n")

        if budget <= 0:
            print("-- break after auction #", index)
            break

        # WON BID ______________________________________________

        if constant_bid >= row['payprice']:     

            bids_won += 1                        # won the bid
            total_paid += row['payprice']        # add amount to total_paid   
            budget = budget - row['payprice']    # substract money from budget

            #if constant_bid == row['bidprice']:      
                #budget = budget - row['payprice']    # substract money from budget

            #elif constant_bid > row['bidprice']:
            #    budget = budget - row['bidprice']    # substract money from budget

            # CLICK = 1 ______________________________________________

            if row['click'] == 1:    # only reduce money from budget if ad has been clicked

                    earned_clicks += 1                   # earn the click
                    #print("current budget:", budget, ", earned clicks:", earned_clicks, "\n")

        #if index%100000 == 0:
        #    print("bid#", index, ", budget:", budget, ", payprice:", row['payprice'], ", bids_won:", bids_won, ", earned_clicks:", earned_clicks, "\n")



    print("__________________________________\n")

    if earned_clicks > 0:
        cpc = total_paid / earned_clicks
    if bids_won > 0:
        ctr = earned_clicks / bids_won

    print("left budget:", budget)
    print("bids_won:", bids_won)
    print("earned clicks:", earned_clicks)
    print("CTR:", ctr)
    print("cost per click:", cpc)




constant_bid: 68 

__________________________________

left budget: 965951
bids_won: 152404
earned clicks: 57
CTR: 0.00037400593160284506
cost per click: 92702.61403508772


constant_bid: 69 

__________________________________

left budget: 897434
bids_won: 153397
earned clicks: 58
CTR: 0.0003781038742609047
cost per click: 92285.62068965517


constant_bid: 70 

-- break after auction # 287980
__________________________________

left budget: 0
bids_won: 162101
earned clicks: 69
CTR: 0.00042566054496887744
cost per click: 90579.71014492754


constant_bid: 71 

-- break after auction # 285790
__________________________________

left budget: 0
bids_won: 161302
earned clicks: 69
CTR: 0.0004277690295222626
cost per click: 90579.71014492754


constant_bid: 72 

-- break after auction # 281812
__________________________________

left budget: 0
bids_won: 160523
earned clicks: 68
CTR: 0.00042361530746372794
cost per click: 91911.76470588235


constant_bid: 73 

-- break after auction # 26715